In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph
# https://blog.0x.org/0x-ecosystem-update-13/

In [2]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import dataframe_image as dfi
import os
from datetime import datetime
import numpy as np
pd.set_option('display.max_rows', 99999)
pd.set_option('display.max_columns', 99999)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def client(url):
    sample_transport = RequestsHTTPTransport(url = url,
                                             verify = True,
                                             retries = 3)
    
    client = Client(transport=sample_transport)
    return client

In [4]:
# interesting tokens on MATIC

 #https://polygon.fulcrum.trade/borrow/asset
lending_tokens_fulcrum = ['LINK', 'BZRX', 'AAVE', 'WBTC', 'USDC', 'USDT', 'WETH', 'WMATIC']

# https://creamfinance.substack.com/p/cream-communications-14
lending_tokens_aave = ['MATIC', 'DAI', 'USDC', 'USDT', 'WETH', 'WBTC', 'AAVE']
lending_tokens_cream = ['USDC', 'USDT', 'DAI', 'WMATIC', 'WETH', 'WBTC', 'LINK', 'SUSHI', 'CRV', 'QUICK']
remaining_interesting_tokens = ['FRAX', 'ibBTC', 'ILV', 'ETH', 'MATIC']

interesting_tokens = list(set(lending_tokens_fulcrum +\
                              lending_tokens_aave +\
                              lending_tokens_cream +\
                              remaining_interesting_tokens))

interesting_tokens

['BZRX',
 'QUICK',
 'ibBTC',
 'WBTC',
 'CRV',
 'AAVE',
 'USDT',
 'FRAX',
 'WMATIC',
 'ILV',
 'LINK',
 'MATIC',
 'ETH',
 'USDC',
 'DAI',
 'SUSHI',
 'WETH']

## Quickswap

In [5]:
query = gql('''
query {
  uniswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sameepsi/quickswap02').execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

7798


In [6]:
try:
    df = pd.read_excel('pairdata_Quickswap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [7]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [8]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel('pairdata_Quickswap_MATIC.xlsx', index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,WMATIC-QUICK,WMATIC,QUICK
1,0x02e0e14d65175420fb12acd325a7be456bc22005,QUICK-DAI,QUICK,DAI
2,0x0554059d42e26f35cc958581c71fdfd92405d02f,WMATIC-AAVE,WMATIC,AAVE
3,0x0b3a76d33d93b2651dbf03b102eefa9500e89243,SUSHI-WMATIC,SUSHI,WMATIC
4,0x1bd06b96dd42ada85fdd0795f3b4a79db914add5,WETH-QUICK,WETH,QUICK


In [9]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [10]:
df_old = pd.read_csv("Quickswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-20 07:35:11.423300,8.944483e+08


In [11]:
df4.to_csv("Quickswap export.csv", index = False)

In [12]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.003 / df_combined['timedelta']
df_combined.head(3)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x019ba0325f1988213d448b3472fa1cf8d07618d7,2021-07-22 06:42:09.936482,8.990493e+08,2021-07-20 07:35:11.423300,8.944483e+08,0.005375,4.600962e+06,2.567981e+06
0x02e0e14d65175420fb12acd325a7be456bc22005,2021-07-22 06:42:10.139152,2.457933e+05,2021-07-20 07:35:11.616338,2.439013e+05,0.005375,1.891923e+03,1.055957e+03
0x0554059d42e26f35cc958581c71fdfd92405d02f,2021-07-22 06:42:10.337685,4.497729e+05,2021-07-20 07:35:11.825907,4.468258e+05,0.005375,2.947144e+03,1.644919e+03


In [13]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/moonster-bsc/quickswap-daypairdataslim')\
                      .execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 47 pair: 0x019ba0325f1988213d448b3472fa1cf8d07618d7
 2 / 47 pair: 0x02e0e14d65175420fb12acd325a7be456bc22005
 3 / 47 pair: 0x0554059d42e26f35cc958581c71fdfd92405d02f
 4 / 47 pair: 0x0b3a76d33d93b2651dbf03b102eefa9500e89243
 5 / 47 pair: 0x1bd06b96dd42ada85fdd0795f3b4a79db914add5
 6 / 47 pair: 0x1f1e4c845183ef6d50e9609f16f6f9cae43bc9cb
 7 / 47 pair: 0x2aa7a18ceabf2ef893d2f7c0145cc45e6f10b223
 8 / 47 pair: 0x2cf7252e74036d1da831d11089d326296e64a728
 9 / 47 pair: 0x3c986748414a812e455dcd5418246b8fded5c369
10 / 47 pair: 0x4a35582a710e1f4b2030a3f826da20bfb6703c09
11 / 47 pair: 0x4f2cc25028d4d80355c802482463b5b2a93cdb84
12 / 47 pair: 0x527a26525f5483612a1f96cfceacc7032b1c8610
13 / 47 pair: 0x59153f27eefe07e5ece4f9304ebba1da6f53ca88
14 / 47 pair: 0x5ca6ca6c3709e1e6cfe74a50cf6b2b6ba2dadd67
15 / 47 pair: 0x600b588d7e847665b2f36cdc70838d04ef9790c2
16 / 47 pair: 0x604229c960e5cacf2aaeac8be68ac07ba9df81c3
17 / 47 pair: 0x609360d5f6dcc26597165ac76e432a16fc27b6fd
18 / 47 pair: 0x634f9d9f8680349

In [14]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['dailyVolumeUSD','reserveUSD']] = df5[['dailyVolumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['dailyVolumeUSD'] * 0.003, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Quickswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 200), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 500), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 1000), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] - 2000), 3)

df5.head(2)

,dailyVolumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,3.982639e+05,2021-07-22,1.634449e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,1194.791677,2.668,2.668,2.668,2.668,2.668,2.668,2.668,2.668,2.668
1,3.091216e+06,2021-07-21,1.630525e+07,0x019ba0325f1988213d448b3472fa1cf8d07618d7,[Quickswap MATIC] WMATIC-QUICK,WMATIC,QUICK,9273.647430,20.759,20.759,20.759,20.758,20.757,20.760,20.760,20.761,20.762


In [15]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,dailyVolumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xf04adbf75cdfc5ed26eea4bbbb991db002036bdd,1.125950e+06,2021-07-21,3.214027e+07,[Quickswap MATIC] USDC-DAI,3377.849959,3.836,3.836,3.836,3.836,3.836,3.836,3.836,3.836,3.836
0x90bc3e68ba8393a3bf2d79309365089975341a43,1.089972e+06,2021-07-21,2.789701e+07,[Quickswap MATIC] WETH-AAVE,3269.917140,4.278,4.278,4.278,4.278,4.278,4.278,4.278,4.278,4.279


In [16]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x019ba0325f1988213d448b3472fa1cf8d07618d7,15.711603,15.711411,15.711122,15.710642,15.709681,15.711795,15.712084,15.712564,15.713526,1.634449e+07
0x02e0e14d65175420fb12acd325a7be456bc22005,18.690912,18.051860,17.171221,15.880070,13.804129,19.376870,20.505713,22.710829,28.933695,5.649577e+03


In [17]:

df_final_quickswap = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'dailyVolumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_quickswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                            
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                            
                              'APR timedelta Liq - 2000',                            
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                                            
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                                            
                              'APR yday Liq - 2000',                              
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_quickswap.head(1)

# df_final_quickswap_stylized = df_final_quickswap.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_quickswap_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Quickswap MATIC] WMATIC-USDT,2021-07-21,80.369007,80.34102,80.299076,80.229268,80.090014,80.397013,80.439059,80.509233,80.64995,100.972,100.937,100.884,100.795,100.62,101.007,101.06,101.149,101.326,574136.120803,571499.614231,526989.72551


In [18]:
df_final_quickswap

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Quickswap MATIC] WMATIC-USDT,2021-07-21,80.369007,8.034102e+01,8.029908e+01,8.022927e+01,8.009001e+01,8.039701e+01,8.043906e+01,8.050923e+01,8.064995e+01,100.972,100.937,100.884,100.795,100.620,101.007,101.060,101.149,101.326,5.741361e+05,5.714996e+05,5.269897e+05
[Quickswap MATIC] WMATIC-USDC,2021-07-21,76.198567,7.619803e+01,7.619722e+01,7.619588e+01,7.619319e+01,7.619911e+01,7.619991e+01,7.620126e+01,7.620395e+01,92.654,92.653,92.652,92.651,92.647,92.655,92.656,92.657,92.661,2.832408e+07,2.768377e+07,2.342478e+07
[Quickswap MATIC] WMATIC-DAI,2021-07-21,61.217781,6.116361e+01,6.108255e+01,6.094791e+01,6.068040e+01,6.127204e+01,6.135361e+01,6.149005e+01,6.176476e+01,72.513,72.449,72.353,72.193,71.876,72.577,72.674,72.835,73.161,2.258396e+05,2.257846e+05,1.495184e+05
[Quickswap MATIC] USDC-QUICK,2021-07-21,35.774755,3.577370e+01,3.577213e+01,3.576950e+01,3.576425e+01,3.577581e+01,3.577738e+01,3.578001e+01,3.578526e+01,46.737,46.736,46.734,46.730,46.723,46.739,46.741,46.744,46.751,6.810123e+06,6.756129e+06,2.883677e+06
[Quickswap MATIC] WMATIC-WBTC,2021-07-21,31.552521,3.148261e+01,3.137832e+01,3.120603e+01,3.086708e+01,3.162274e+01,3.172867e+01,3.190679e+01,3.226910e+01,38.750,38.664,38.535,38.322,37.904,38.836,38.967,39.187,39.634,9.006431e+04,8.968679e+04,3.173825e+04
[Quickswap MATIC] FRAX-QUICK,2021-07-21,28.460205,2.834952e+01,2.818510e+01,2.791527e+01,2.739082e+01,2.857175e+01,2.874073e+01,2.902684e+01,2.961649e+01,25.854,25.758,25.616,25.381,24.925,25.951,26.098,26.346,26.856,5.122704e+04,5.362895e+04,1.266247e+04
NaN,NaN,27.142149,2.714209e+01,2.714199e+01,2.714184e+01,2.714153e+01,2.714221e+01,2.714230e+01,2.714246e+01,2.714277e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.782661e+07,NaN,NaN
[Quickswap MATIC] QUICK-USDT,2021-07-21,24.104947,2.391477e+01,2.363506e+01,2.318315e+01,2.232926e+01,2.429817e+01,2.459389e+01,2.510308e+01,2.618745e+01,28.636,28.412,28.081,27.548,26.540,28.864,29.213,29.813,31.091,2.514994e+04,2.532724e+04,6.623446e+03
[Quickswap MATIC] USDC-WETH,2021-07-21,20.224774,2.022475e+01,2.022470e+01,2.022463e+01,2.022450e+01,2.022480e+01,2.022484e+01,2.022491e+01,2.022505e+01,23.658,23.658,23.658,23.658,23.658,23.658,23.658,23.659,23.659,1.454376e+08,1.440699e+08,3.112751e+07


In [19]:
# dfi.export(df_final_quickswap_stylized, 'Quickswap MATIC timedelta.png')
# os.system('xdg-open "Quickswap MATIC timedelta.png"')

## Sushiswap MATIC

In [20]:
# sushiswap_matic = 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange'

# sample_transport=RequestsHTTPTransport(url = sushiswap_matic,
#                                        verify = True,
#                                        retries = 3)

# client = Client(transport=sample_transport)

In [21]:
query = gql('''
query {
  factories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

1688


In [22]:
try:
    df = pd.read_excel('pairdata_SushiSwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client.execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [23]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [24]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_SushiSwap_MATIC.xlsx", index = False)
df_filtered.head()

,id,Pair,Token 0,Token 1
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,WMATIC-CRV,WMATIC,CRV
1,0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,SUSHI-DAI,SUSHI,DAI
2,0x264a05020b491ba1969a325b0a474a864bb0d93b,FRAX-USDT,FRAX,USDT
3,0x2813d43463c374a680f235c428fb1d7f08de0b69,WETH-AAVE,WETH,AAVE
4,0x29bb3ddf61c831adda06439740b340ae48606da6,WBTC-QUICK,WBTC,QUICK


In [25]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df4 = df4.append(df1)
    
df4 = df4[['id', 'timestamp', 'volumeUSD']]
df4.head(1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

,id,timestamp,volumeUSD
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-22 06:44:29.892456,2832.473936


In [26]:
df_old = pd.read_csv("SushiSwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-20 07:37:10.699927,1434.537444


In [27]:
df4.to_csv("SushiSwap export.csv", index = False)

In [28]:
df_combined = pd.concat([df4.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.0025 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,2021-07-22 06:44:29.892456,2832.473936,2021-07-20 07:37:10.699927,1434.537444,0.005376,1397.936493,780.148988
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,2021-07-22 06:44:30.100002,10764.997505,2021-07-20 07:37:10.923506,10660.566960,0.005376,104.430545,58.279752


In [29]:
df4 = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pair:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    volumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df4 = df4.append(df1)

 1 / 44 pair: 0x140ea3fae80a2732ebc4de0511ff84ef1a575217
 2 / 44 pair: 0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05
 3 / 44 pair: 0x264a05020b491ba1969a325b0a474a864bb0d93b
 4 / 44 pair: 0x2813d43463c374a680f235c428fb1d7f08de0b69
 5 / 44 pair: 0x29bb3ddf61c831adda06439740b340ae48606da6
 6 / 44 pair: 0x34965ba0ac2451a34a0471f04cca3f990b8dea27
 7 / 44 pair: 0x383be588327216586e131e63592a2dc976a16655
 8 / 44 pair: 0x396e655c309676caf0acf4607a868e0cded876db
 9 / 44 pair: 0x3b31bb4b6ba4f67f4ef54e78bcb0aaa4f53dc7ff
10 / 44 pair: 0x3bffd4ca16197ad48dc610c41a022664905b7ffc
11 / 44 pair: 0x3e9a0586088caeb2adff32b0c06cfa44151c7a5b
12 / 44 pair: 0x4b1f1e2435a9c96f7330faea190ef6a7c8d70001
13 / 44 pair: 0x55ff76bffc3cdd9d5fdbbc2ece4528ecce45047e
14 / 44 pair: 0x597a9bc3b24c2a578ccb3aa2c2c62c39427c6a49
15 / 44 pair: 0x5c26e61c6153805b073beb185319de61adbdcb98
16 / 44 pair: 0x5f3eeceead64ee34b78e80cd574496e82e537541
17 / 44 pair: 0x68cce7049013ca8df91cd512cefee8fc0bb8d926
18 / 44 pair: 0x6ff62bfb8c12109

In [30]:
df5 = df4.merge(df_filtered,
                left_on = 'id',
                right_on = 'id',
                how = 'left',
                validate = 'many_to_one')
df5[['volumeUSD','reserveUSD']] = df5[['volumeUSD','reserveUSD']].astype(float)
df5['date'] = pd.to_datetime(df5['date'], unit='s').dt.date
df5['fee'] = round(df5['volumeUSD'] * 0.0025, 6)
df5['APR yday'] = round(100 * df5['fee'] * 365 / df5['reserveUSD'], 3)
df5['Pair'] = '[Sushiswap MATIC] ' + df5['Pair']
df5['APR yday Liq + 200'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 200), 3)
df5['APR yday Liq + 500'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 500), 3)
df5['APR yday Liq + 1000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 1000), 3)
df5['APR yday Liq + 2000'] = round(100 * df5['fee'] * 365 / (df5['reserveUSD'] + 2000), 3)
df5['APR yday Liq - 200'] = round(100 * df5['fee'] * 365 / max(df5['reserveUSD'] - 200, 0), 3)
df5['APR yday Liq - 500'] = round(100 * df5['fee'] * 365 / max(df5['reserveUSD'] - 500, 0), 3)
df5['APR yday Liq - 1000'] = round(100 * df5['fee'] * 365 / max(df5['reserveUSD'] - 1000, 0), 3)
df5['APR yday Liq - 2000'] = round(100 * df5['fee'] * 365 / max(df5['reserveUSD'] - 2000, 0), 3)

df5.head(2)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-22,8990.155272,249.174625,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,0.747524,3.035,2.969,2.875,2.731,2.483,3.104,3.214,3.415,3.903
1,2021-07-21,8900.236889,1128.441469,0x140ea3fae80a2732ebc4de0511ff84ef1a575217,[Sushiswap MATIC] WMATIC-CRV,WMATIC,CRV,3.385324,13.883,13.578,13.145,12.481,11.336,14.202,14.710,15.641,17.907


In [31]:
latest_results = df5.sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['Pair'], keep = 'last')\
                    .reset_index(drop = True)\
                    .drop(columns = ['Token 0', 'Token 1'])\
                    .set_index('id')

latest_results.head(2)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xb5846453b67d0b4b4ce655930cf6e4129f4416d7,2021-07-21,1.059788e+06,283246.853406,[Sushiswap MATIC] SUSHI-WETH,849.740560,29.266,29.260,29.252,29.238,29.211,29.271,29.28,29.293,29.321
0x383be588327216586e131e63592a2dc976a16655,2021-07-21,7.396128e+02,816.872274,[Sushiswap MATIC] WMATIC-QUICK,2.450617,120.938,95.196,72.158,51.418,32.650,165.762,373.30,-343.517,-70.968


In [32]:
df_combined2 = pd.concat([df_combined,
                          df5.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / max(df_combined2['reserveUSD'] - 200, 0)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / max(df_combined2['reserveUSD'] - 500, 0)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / max(df_combined2['reserveUSD'] - 1000, 0)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / max(df_combined2['reserveUSD'] - 2000, 0)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x140ea3fae80a2732ebc4de0511ff84ef1a575217,8.677814,8.488964,8.220614,7.809178,7.098617,8.875258,9.188866,9.763878,11.160682,8990.155272
0x177e42f6b3da3ffdda3dcbb6010f00a380c3cb05,12.889840,8.936733,6.120941,4.013378,2.376692,23.114309,-121.763973,-10.637653,-3.765175,452.137114


In [33]:
df_final_sushiswap_matic = pd.concat([df_combined2, latest_results],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                             
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                                'APR timedelta Liq + 500',
                                'APR timedelta Liq + 1000',                                            
                                'APR timedelta Liq + 2000',
                                'APR timedelta Liq - 200',
                                'APR timedelta Liq - 500',
                                'APR timedelta Liq - 1000',                                            
                                'APR timedelta Liq - 2000',                           
                              'APR yday',
                             'APR yday Liq + 200',
                                'APR yday Liq + 500',
                                'APR yday Liq + 1000',                                            
                                'APR yday Liq + 2000',
                                'APR yday Liq - 200',
                                'APR yday Liq - 500',
                                'APR yday Liq - 1000',                                            
                                'APR yday Liq - 2000',                          
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_matic.head(1)

# df_final_sushiswap_matic_stylized = df_final_sushiswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_sushiswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Sushiswap MATIC] WMATIC-USDC,2021-07-21,88.148322,88.143914,88.137303,88.126287,88.104263,88.15273,88.159344,88.170368,88.192425,103.661,103.656,103.648,103.635,103.609,103.667,103.674,103.687,103.713,3.999355e+06,3.984405e+06,3.771952e+06


In [34]:
# dfi.export(df_final_stylized_sushi, 'SushiSwap MATIC timedelta.png')
# os.system('xdg-open "SushiSwap MATIC timedelta.png"')

## Honeyswap Polygon

In [35]:
query = gql('''
query {
  honeyswapFactories {
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(query)
number_of_pairs = int(response['honeyswapFactories'][0]['pairCount'])
print(number_of_pairs)

148


In [36]:
try:
    df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0 {symbol}
                          token1 {symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [37]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [38]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_HoneySwap_MATIC.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x0c787944946d22922c9f41c477cc539700d35bb2,WETH-AAVE,WETH,AAVE
1,0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,WETH-USDT,WETH,USDT


In [39]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

,id,timestamp,volumeUSD
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-22 06:44:50.318729,126034.554001


In [40]:
df_old = pd.read_csv("HoneySwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-20 07:37:29.666656,115814.09095


In [41]:
df_timedelta.to_csv("HoneySwap export.csv", index = False)

In [42]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * 0.0025 / df_combined['timedelta']
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x0c787944946d22922c9f41c477cc539700d35bb2,2021-07-22 06:44:50.318729,126034.554001,2021-07-20 07:37:29.666656,115814.090950,0.005376,10220.463051,4753.086274
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,2021-07-22 06:44:50.530957,222180.722272,2021-07-20 07:37:29.874856,209566.120253,0.005376,12614.602020,5866.494501


In [43]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

 1 / 27 pair: 0x0c787944946d22922c9f41c477cc539700d35bb2
 2 / 27 pair: 0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f
 3 / 27 pair: 0x1fc4a2523349bd6df30000b923bb1acb3a27051f
 4 / 27 pair: 0x2177ccb01ec6a3a4d567e42d51f85a65e3ac5ab6
 5 / 27 pair: 0x218e468b15469228f35b0e7f88425cd45fb982bd
 6 / 27 pair: 0x287e9a601a257b24f8d718d08612e9d8ead7f5a8
 7 / 27 pair: 0x35fbfcd7256b3d6bffc69b407e42a135280c0a8b
 8 / 27 pair: 0x3a968d1fe0246de743b6f102e679c11be36688a9
 9 / 27 pair: 0x3f24e142fbf05d16ab1cf79e6df3473f515b16e0
10 / 27 pair: 0x593a7e3c4897834cb6499e051a1ef9532ca82871
11 / 27 pair: 0x5c09be7481c9341a0e4a04843a936429691571a8
12 / 27 pair: 0x6815c440c5c8ad500e1ac6c301f8057b84753268
13 / 27 pair: 0x6d3842ab227a0436a6e8c459e93c74bd8c16fb34
14 / 27 pair: 0x707c48f37a2dacdb729a6880e1c94768d56bbbb0
15 / 27 pair: 0x7fdf6932d8e761c75922af0d4acd04bed36a8c8b
16 / 27 pair: 0x86b7249272fabb82ef36550ef898ea539225e7f0
17 / 27 pair: 0x8df8b187b9a2a7c2d25faa4f33d06c35b25b28bc
18 / 27 pair: 0x96aa3911e89da7b

In [44]:
df_last_two_days2 = df_last_two_days.merge(df_filtered,
                             left_on = 'id',
                             right_on = 'id',
                             how = 'left',
                             validate = 'many_to_one')
df_last_two_days2[['volumeUSD','reserveUSD']] = df_last_two_days2[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days2['date'] = pd.to_datetime(df_last_two_days2['date'], unit='s').dt.date
df_last_two_days2['fee'] = round(df_last_two_days2['volumeUSD'] * 0.0025, 6)
df_last_two_days2['APR yday'] = round(100 * df_last_two_days2['fee'] * 365 / df_last_two_days2['reserveUSD'], 3)
df_last_two_days2['Pair'] = '[Honeyswap MATIC] ' + df_last_two_days2['Pair']
df_last_two_days2['APR yday Liq + 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 200), 3)
df_last_two_days2['APR yday Liq + 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 500), 3) 
df_last_two_days2['APR yday Liq + 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 1000), 3)                                                  
df_last_two_days2['APR yday Liq + 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] + 2000), 3)
df_last_two_days2['APR yday Liq - 200'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 200), 3)
df_last_two_days2['APR yday Liq - 500'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 500), 3)
df_last_two_days2['APR yday Liq - 1000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 1000), 3)                                                 
df_last_two_days2['APR yday Liq - 2000'] = round(100 * df_last_two_days2['fee'] * 365 / (df_last_two_days2['reserveUSD'] - 2000), 3)                                                 

df_last_two_days2.head(2)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,1550.771538,2021-07-22,147023.633465,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,3.876929,0.962,0.961,0.959,0.956,0.950,0.964,0.966,0.969,0.976
1,3671.726996,2021-07-21,143524.447144,0x0c787944946d22922c9f41c477cc539700d35bb2,[Honeyswap MATIC] WETH-AAVE,WETH,AAVE,9.179317,2.334,2.331,2.326,2.318,2.302,2.338,2.343,2.351,2.367


In [45]:
df_yesterday = df_last_two_days2.sort_values(by = 'date',
                                             ascending = False)\
                                .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                .reset_index(drop = True)\
                                .drop(columns = ['Token 0', 'Token 1'])\
                                .set_index('id')

df_yesterday.head(2)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xd862db749534d539713b2c392421fe5a8e43e19e,20303.557286,2021-07-21,175271.496837,[Honeyswap MATIC] USDC-WETH,50.758893,10.570,10.558,10.540,10.510,10.451,10.583,10.601,10.631,10.692
0xe49fea624d480a5233b5dfc4969a27319873e6f0,78.847796,2021-07-21,18780.147195,[Honeyswap MATIC] USDC-DAI,0.197119,0.383,0.379,0.373,0.364,0.346,0.387,0.394,0.405,0.429


In [46]:
df_combined2 = pd.concat([df_combined,
                          df_last_two_days2.sort_values(by = 'date', ascending = False)\
                             .drop_duplicates(subset = ['Pair'],
                                              keep = 'first')\
                             .set_index('id')['reserveUSD']],
                         axis = 1)
df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',
                             'APR timedelta Liq - 2000',                              
                             'reserveUSD']]
df_combined2.head(2)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x0c787944946d22922c9f41c477cc539700d35bb2,3.232872,3.228480,3.221915,3.211032,3.189485,3.237276,3.243904,3.255012,3.277456,147023.633465
0x1013ba833071fd8aca8bf2ac83e5515c5fb9e76f,8.153249,8.130649,8.096983,8.041489,7.932750,8.175975,8.210302,8.268160,8.386356,71952.842861


In [47]:
df_final_honeyswap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_honeyswap_matic.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_honeyswap_matic.head(1)

# df_final_honeyswap_matic_stylized = df_final_honeyswap_matic.dropna()\
#                                                 .style.format({'APR timedelta': "{:0<4,.2f}",
#                                                                'APR timedelta Liq + 100': "{:0<4,.2f}",
#                                                            'APR yday': "{:0<4,.2f}",
#                                                            'APR yday Liq + 100': "{:0<4,.2f}",
#                                                            'Liquidity timedelta': "{:0<4,.0f}",
#                                                            'Liquidity yday': "{:0<4,.0f}",
#                                                            'Volume yday': "{:0<4,.0f}"})

# df_final_honeyswap_matic_stylized

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Honeyswap MATIC] WMATIC-WETH,2021-07-21,14.944321,14.930456,14.909708,14.875254,14.806823,14.958211,14.979096,15.014032,15.084397,18.046,18.03,18.004,17.963,17.879,18.063,18.089,18.131,18.217,215374.653299,214164.690022,42355.182516


## DoDoSwap Polygon

In [48]:
client_exe = client('https://api.thegraph.com/subgraphs/name/dodoex/dodoex-v2-polygon')
# https://thegraph.com/legacy-explorer/subgraph/dodoex/dodoex-v2-polygon
fee = 0.0025

In [49]:
query = gql('''
query {
  dodoZoos {
    pairCount
  }
}
''')

response = client_exe.execute(query)
number_of_pairs = int(response['dodoZoos'][0]['pairCount'])
print(number_of_pairs)

257


In [50]:
# # try:
# #     df = pd.read_excel('pairdata_HoneySwap_MATIC.xlsx')
# # except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      baseSymbol
                      quoteSymbol
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass

100
200
300


In [51]:
df = df.dropna()


df['Pair'] = df['baseSymbol'] + '-' + df['quoteSymbol']
df.columns = ['Token 0', 'id', 'Token 1', 'Pair']
df = df[['id','Pair', 'Token 0', 'Token 1']]
df.head(2)

,id,Pair,Token 0,Token 1
0,0x0000ff310eec299c469e5b1fabff663b7ee5b100,zMATIC-WMATIC,zMATIC,WMATIC
1,0x01577045e19463645a3043094bce5bc90291a51d,ONE-WMATIC,ONE,WMATIC


In [52]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_DoDoSwap_MATIC.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
55,0x0f20c4148369fb70083593b1475922bec87aebe3,WMATIC-USDC,WMATIC,USDC
64,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,WMATIC-USDC,WMATIC,USDC


In [53]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
        lpFeeRate
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD', 'lpFeeRate']]
df_timedelta.head(1)

 1 / 9 pair: 0x0f20c4148369fb70083593b1475922bec87aebe3
 2 / 9 pair: 0x10dd6d8a29d489bede472cc1b22dc695c144c5c7
 3 / 9 pair: 0x1ea658dd919d77edccea1a7bfd417b048934ad6b
 4 / 9 pair: 0x2f7e940ea475cb424c8ba6025d48350faea48dc1
 5 / 9 pair: 0x460006fd8244a5c0bcaa4dbb02a0c201db5572bf
 6 / 9 pair: 0x4b1beedb18156148c58cbf620279d06268132586
 7 / 9 pair: 0x5333eb1e32522f1893b7c9fea3c263807a02d561
 8 / 9 pair: 0x59c1b9239abcbe64b62fcd172c0c65ef34ea02c2
 9 / 9 pair: 0x63b3581aedd3737a544e0775930d30d4a0241ccb


,id,timestamp,volumeUSD,lpFeeRate
0,0x0f20c4148369fb70083593b1475922bec87aebe3,2021-07-22 06:45:05.792064,66.169014,0.004


In [54]:
df_old = pd.read_csv("DoDoSwap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD', 'lpFeeRate'])
df_old = df_old[['id', 'timestamp', 'volumeUSD', 'lpFeeRate']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old', 'lpFeeRate']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old,lpFeeRate
0,0x0f20c4148369fb70083593b1475922bec87aebe3,2021-07-20 07:37:44.261196,66.169014,0.004


In [55]:
df_timedelta.to_csv("DoDoSwap export.csv", index = False)

In [56]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old').drop(columns = ['lpFeeRate'])], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['lpFeeRate'] = df_combined['lpFeeRate'].astype(float)
df_combined['fees per year'] = df_combined['volumeUSD delta'] * df_combined['lpFeeRate'] / df_combined['timedelta']
df_combined = df_combined.drop(columns = ['lpFeeRate'])
df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x0f20c4148369fb70083593b1475922bec87aebe3,2021-07-22 06:45:05.792064,6.616901e+01,2021-07-20 07:37:44.261196,6.616901e+01,0.005376,0.000000e+00,0.000000e+00
0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,2021-07-22 06:45:05.991589,1.365076e+07,2021-07-20 07:37:44.455454,7.505972e+06,0.005376,6.144790e+06,1.714593e+06


In [57]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    pairAddress
    volumeUSD
    baseTokenReserve
    quoteTokenReserve
    baseUsdPrice
    quoteUsdPrice
    lpFeeRate
  }
}'''
    response = client_exe.execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['baseTokenReserve',
                  'baseUsdPrice',
                  'lpFeeRate',
                  'quoteTokenReserve',
                  'quoteUsdPrice',
                  'volumeUSD']] = \
df_last_two_days[['baseTokenReserve',
                  'baseUsdPrice',
                  'lpFeeRate',
                  'quoteTokenReserve',
                  'quoteUsdPrice',
                  'volumeUSD']].astype(float)

df_last_two_days['reserveUSD'] = df_last_two_days['baseTokenReserve'] * df_last_two_days['baseUsdPrice'] + df_last_two_days['quoteTokenReserve'] * df_last_two_days['quoteUsdPrice']
df_last_two_days = df_last_two_days.drop(columns = ['baseTokenReserve',
                                                    'baseUsdPrice',
                                                    'quoteTokenReserve',
                                                    'quoteUsdPrice'])
df_last_two_days

 1 / 9 pair: 0x0f20c4148369fb70083593b1475922bec87aebe3
 2 / 9 pair: 0x10dd6d8a29d489bede472cc1b22dc695c144c5c7
 3 / 9 pair: 0x1ea658dd919d77edccea1a7bfd417b048934ad6b
 4 / 9 pair: 0x2f7e940ea475cb424c8ba6025d48350faea48dc1
 5 / 9 pair: 0x460006fd8244a5c0bcaa4dbb02a0c201db5572bf
 6 / 9 pair: 0x4b1beedb18156148c58cbf620279d06268132586
 7 / 9 pair: 0x5333eb1e32522f1893b7c9fea3c263807a02d561
 8 / 9 pair: 0x59c1b9239abcbe64b62fcd172c0c65ef34ea02c2
 9 / 9 pair: 0x63b3581aedd3737a544e0775930d30d4a0241ccb


,date,lpFeeRate,pairAddress,volumeUSD,id,reserveUSD
0,1626134400,0.00400,0x0f20c4148369fb70083593b1475922bec87aebe3,6.616901e+01,0x0f20c4148369fb70083593b1475922bec87aebe3,742.454507
0,1626912000,0.00150,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,7.096271e+05,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,41994.107215
1,1626825600,0.00120,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,3.795330e+06,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,40350.790837
0,1626825600,0.00800,0x1ea658dd919d77edccea1a7bfd417b048934ad6b,1.886039e-03,0x1ea658dd919d77edccea1a7bfd417b048934ad6b,0.056496
1,1626739200,0.00800,0x1ea658dd919d77edccea1a7bfd417b048934ad6b,1.459609e-03,0x1ea658dd919d77edccea1a7bfd417b048934ad6b,0.052081
0,1626912000,0.00300,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,5.903866e-02,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,3.074376
1,1626825600,0.00300,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,4.942081e-01,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,1.349583
0,1626912000,0.00088,0x460006fd8244a5c0bcaa4dbb02a0c201db5572bf,3.532696e-02,0x460006fd8244a5c0bcaa4dbb02a0c201db5572bf,10.168077
1,1626825600,0.00088,0x460006fd8244a5c0bcaa4dbb02a0c201db5572bf,3.452736e-01,0x460006fd8244a5c0bcaa4dbb02a0c201db5572bf,10.162861
0,1626652800,0.00300,0x4b1beedb18156148c58cbf620279d06268132586,8.298601e-05,0x4b1beedb18156148c58cbf620279d06268132586,0.001917


In [58]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                             left_on = 'id',
                             right_on = 'id',
                             how = 'left',
                             validate = 'many_to_one')
df_last_two_days_incl_pairs[['volumeUSD','reserveUSD']] = df_last_two_days_incl_pairs[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days_incl_pairs['date'] = pd.to_datetime(df_last_two_days_incl_pairs['date'], unit='s').dt.date
df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * df_last_two_days_incl_pairs['lpFeeRate'], 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[DodoSwap MATIC] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3) 
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)                                                  
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)                                                 
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)                                                 

df_last_two_days_incl_pairs.head(2)

,date,lpFeeRate,pairAddress,volumeUSD,id,reserveUSD,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-13,0.0040,0x0f20c4148369fb70083593b1475922bec87aebe3,66.169014,0x0f20c4148369fb70083593b1475922bec87aebe3,742.454507,[DodoSwap MATIC] WMATIC-USDC,WMATIC,USDC,0.264676,13.012,10.251,7.775,5.544,3.523,17.809,39.845,-37.511,-7.682
1,2021-07-22,0.0015,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,709627.138715,0x10dd6d8a29d489bede472cc1b22dc695c144c5c7,41994.107215,[DodoSwap MATIC] WMATIC-USDC,WMATIC,USDC,1064.440708,925.179,920.794,914.293,903.661,883.120,929.607,936.328,947.748,971.445


In [59]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,date,lpFeeRate,pairAddress,volumeUSD,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,,,
0x2f7e940ea475cb424c8ba6025d48350faea48dc1,2021-07-21,0.003,0x2f7e940ea475cb424c8ba6025d48350faea48dc1,0.494208,1.349583,[DodoSwap MATIC] WMATIC-WBTC,0.001483,40.108,0.269,0.108,0.054,0.027,-0.272,-0.109,-0.054,-0.027


In [60]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_DoDoSwap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_DoDoSwap_matic.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_DoDoSwap_matic.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
NaN,NaN,4082.937998,4063.584878,4034.896772,3987.972937,3897.325048,4102.476341,4132.137008,4182.536166,4287.114977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41994.107215,NaN,NaN


## Ape Swap Polygon

In [61]:
client_exe = client('https://api.thegraph.com/subgraphs/name/apeswapfinance/dex-polygon') # https://thegraph.com/legacy-explorer/subgraph/apeswapfinance/dex-polygon
fee = 0.0005

query = gql('''
            query {uniswapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:


193


In [62]:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

95
95


,id,token0,token1
0,0x0179a46400213c57b7d2abc7a8768a5a64a190ec,{'symbol': 'BIBLE'},{'symbol': 'USDT'}


In [63]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)



,id,Pair,Token 0,Token 1
1,0x019011032a7ac3a87ee885b6c08467ac46ad11cd,WMATIC-USDC,WMATIC,USDC


In [64]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 20 pair: 0x019011032a7ac3a87ee885b6c08467ac46ad11cd
 2 / 20 pair: 0x237ac473ac0ae4551019d9298a4118b3144f26a8
 3 / 20 pair: 0x39a9b6474db467c2874db622a0e048013489501c
 4 / 20 pair: 0x42ed6d85ccf43859cbc46f6efa1f21e21cc24030
 5 / 20 pair: 0x5b13b583d4317ab15186ed660a1e4c65c10da659
 6 / 20 pair: 0x65d43b64e3b31965cd5ea367d4c2b94c03084797
 7 / 20 pair: 0x6cf8654e85ab489ca7e70189046d507eba233613
 8 / 20 pair: 0x7433afe84df37d0954ff87d7f5788f124f8597f8
 9 / 20 pair: 0x7b2dd4bab4487a303f716070b192543ea171d3b2
10 / 20 pair: 0x84964d9f9480a1db644c2b2d1022765179a40f68
11 / 20 pair: 0x8b6631324d4bb1759f708dde77e93ba898bb58c4
12 / 20 pair: 0x8bf34ff5945d519cf3eb4360e97585605c99daa6
13 / 20 pair: 0x9ec257c1862f1bdf0603a6c20ed6f3d6bae6deb0
14 / 20 pair: 0xa8eca6cc6fb9f8cfa9d3b17d4997cce79e5110cf
15 / 20 pair: 0xab291e39cd0f8235ba42a20a86f996a3f8b934e1
16 / 20 pair: 0xbff538ad7c1fd067f5529bbc7aa3b403f66d70cf
17 / 20 pair: 0xcdbcc1ac4d4e18fb1f2f2d604144fd33e77cda52
18 / 20 pair: 0xd32f3139a214034

,id,timestamp,volumeUSD
0,0x019011032a7ac3a87ee885b6c08467ac46ad11cd,2021-07-22 06:45:16.517542,628291.003158


In [65]:
df_old = pd.read_csv("ApeSwap export MATIC.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

df_timedelta.to_csv("ApeSwap export MATIC.csv", index = False)

df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 20 pair: 0x019011032a7ac3a87ee885b6c08467ac46ad11cd
 2 / 20 pair: 0x237ac473ac0ae4551019d9298a4118b3144f26a8
 3 / 20 pair: 0x39a9b6474db467c2874db622a0e048013489501c
 4 / 20 pair: 0x42ed6d85ccf43859cbc46f6efa1f21e21cc24030
 5 / 20 pair: 0x5b13b583d4317ab15186ed660a1e4c65c10da659
 6 / 20 pair: 0x65d43b64e3b31965cd5ea367d4c2b94c03084797
 7 / 20 pair: 0x6cf8654e85ab489ca7e70189046d507eba233613
 8 / 20 pair: 0x7433afe84df37d0954ff87d7f5788f124f8597f8
 9 / 20 pair: 0x7b2dd4bab4487a303f716070b192543ea171d3b2
10 / 20 pair: 0x84964d9f9480a1db644c2b2d1022765179a40f68
11 / 20 pair: 0x8b6631324d4bb1759f708dde77e93ba898bb58c4
12 / 20 pair: 0x8bf34ff5945d519cf3eb4360e97585605c99daa6
13 / 20 pair: 0x9ec257c1862f1bdf0603a6c20ed6f3d6bae6deb0
14 / 20 pair: 0xa8eca6cc6fb9f8cfa9d3b17d4997cce79e5110cf
15 / 20 pair: 0xab291e39cd0f8235ba42a20a86f996a3f8b934e1
16 / 20 pair: 0xbff538ad7c1fd067f5529bbc7aa3b403f66d70cf
17 / 20 pair: 0xcdbcc1ac4d4e18fb1f2f2d604144fd33e77cda52
18 / 20 pair: 0xd32f3139a214034

,volumeUSD,date,reserveUSD,id
0,305.127345,2021-07-22,1668.110036,0x019011032a7ac3a87ee885b6c08467ac46ad11cd


In [66]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[ApeSwap MATIC] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / max(df_last_two_days_incl_pairs['reserveUSD'] - 200, 0), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / max(df_last_two_days_incl_pairs['reserveUSD'] - 500, 0), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / max(df_last_two_days_incl_pairs['reserveUSD'] - 1000, 0), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / max(df_last_two_days_incl_pairs['reserveUSD'] - 2000, 0), 3)

df_last_two_days_incl_pairs.head(1)

df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_apeswap_matic = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_apeswap_matic.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_apeswap_matic.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[ApeSwap MATIC] WMATIC-USDC,2021-07-21,146.048964,130.412951,112.367795,91.310231,66.41724,165.945152,208.564036,364.64913,-734.055769,92.482,82.563,71.121,57.776,42.01,105.111,132.185,231.621,-459.123,1668.110036,1664.679251,2108.952319


## Balancer Polygon

In [67]:
# client_exe = client('https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-polygon-v2')
# #https://thegraph.com/explorer/subgraph/balancer-labs/balancer-polygon-v2
# fee = 0.0025

In [68]:
# query = gql('''
#             query {
#                   balancers {
#                     poolCount
#                   }
#                 }
#             ''')

# response = client_exe.execute(query)
# number_of_pairs = int(response['balancers'][0]['poolCount'])
# print(number_of_pairs)

In [69]:
# # make a list of ids and pairs on balancer

# df = pd.DataFrame([])
# response_dict = {}
# response_list = []


# for i in range(number_of_pairs):
#     if i % 100 == 0:
#         querystring = '''
#               query {
#               pools(first: '''+str(100)+' skip: '+str(i)+'''){
#                       tokens{symbol}
#                       id
#                   }
#                 }
#                 '''
#         try:
#             response = client_exe.execute(gql(querystring))
#             df = df.append(pd.DataFrame.from_dict(response['pools']))
#             print(len(df))
#         except:
#             pass
    
# df['tokens'] = df['tokens'].astype(str).str.replace("{'symbol': ",'', regex= True)\
#                                            .replace("}",'', regex= True)\
#                                            .replace("]",'', regex= True)\
#                                            .replace('\[', '', regex = True)\
#                                            .replace("'",'', regex= True)\
#                                            .replace(" ",'', regex= True)

# number_different_tokens = len(df['tokens'].str.split(',', expand = True).columns)
# column_list = []

# for token_column in range(number_different_tokens):
#     column_list.append('Token' + str(token_column + 1))
    
# df[column_list] = df['tokens'].str.split(',', expand = True)
# df.replace()

In [70]:
# # make a list of all tokens on Balancer that are not in my interesting tokens

# df_tokens = pd.DataFrame([])

# for column in column_list:
#     df_tokenpart = df[[column]]
#     df_tokenpart = df_tokenpart.rename(columns = {column:'Token'})
#     df_tokens = df_tokens.append(df_tokenpart)
    
# all_balancer_tokens = df_tokens.drop_duplicates().dropna().sort_values(by = 'Token')['Token'].unique().tolist()
# tokens_to_avoid = set(all_balancer_tokens) - set(interesting_tokens)

In [71]:
# # filter out the pairs on balancer that contain tokens to avoid
# df_filtered = df.copy()

# for column in column_list:
#     df_filtered = df_filtered.loc[~df[column].isin(tokens_to_avoid)]
    
# df_filtered = df_filtered[['id', 'tokens']]
# df_filtered

In [72]:
# df_timedelta = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')

#     querystring = '''
#                   query {
#       pools(where: {id:"''' + id + '''"}){
#         totalSwapVolume
#       }
#     }'''
#     response = client_exe.execute(gql(querystring))

#     df1 = pd.DataFrame.from_dict(response['pools'])
#     df1['id'] = id
#     df1['timestamp'] = datetime.now()
#     df1['volumeUSD'] = df1['totalSwapVolume'].astype(float)
#     df_timedelta = df_timedelta.append(df1)
    
# df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
# df_timedelta.head(1)

In [73]:
# df_old = pd.read_csv("Balancer export MATIC.csv",
#                      usecols = ['id', 'timestamp', 'volumeUSD'])
# df_old = df_old[['id', 'timestamp', 'volumeUSD']]
# df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
# df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
# df_old.head(1)

In [74]:
# df_timedelta.to_csv("Balancer export MATIC.csv", index = False)

In [75]:
# df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
# df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
# df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
# df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

# df_combined.head(2)

In [76]:
# querystring = """
#               query {
#                     poolSnapshots(orderBy: timestamp, orderDirection: desc){
#                       timestamp
#                     }
#                     }
#                 """

# response = client_exe.execute(gql(querystring))

# df_time = pd.DataFrame.from_dict(response['poolSnapshots'])\
#                       .drop_duplicates()\
#                       .sort_values(by = 'timestamp', ascending = False).head(2)

# df_time['timestamp'].values.tolist()

In [77]:
# {
#   pools{
#     id
#     totalSwapVolume
#     totalLiquidity
#   }
# }

In [78]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   amounts
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
# }
# }
# """

In [79]:
# """
# {
# poolSnapshots(where: {id:"0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008-1625270400"} orderBy: timestamp, orderDirection: desc first: 2){
#   totalShares
#   swapVolume
#   swapFees
#   timestamp
#   pool{historicalValues(orderBy: block, orderDirection: desc first: 2){
#     poolLiquidity
#     poolId {
#       id
#     }
#     block
#     id
#   }}
# }
# }
# """

In [80]:
# df_last_two_days = pd.DataFrame([])
# lenid = len(df_filtered['id'])

# for count, id in enumerate(df_filtered['id']):
#     print(f'{count+1:2} / {lenid} pair: {id}')
        
#     querystring = '''
#               query {
#   pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
#     date
#     dailyVolumeUSD
#     reserveUSD
#   }
# }'''
#     response = client('https://api.thegraph.com/subgraphs/name/1hive/honeyswap-polygon').execute(gql(querystring))
#     response
    
#     df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
#     df1['id'] = id
#     df_last_two_days = df_last_two_days.append(df1)
    
# df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})

## FTM settings

In [81]:
lending_tokens_cream = ['USDC', 'WFTM', 'DAI', 'ETH', 'BTC', 'LINK', 'SUSHI', 'YFI', 'SNX', 'BAND', 'AAVE', 'COVER', 'CREAM', 'HEGIC']
remaining_interesting_tokens = ['WFTM', 'FTM', 'USDC', 'DAI', 'ETH', 'fUSDT', 'FETH', 'WBTC', 'FRAX', 'FRAX', 'ibBTC']

interesting_tokens = list(set(lending_tokens_cream +\
                              remaining_interesting_tokens))

## SpiritSwap

In [ ]:
fee = 0.0025 #0.3% fee, 0.05% goes to treasury. https://layer3.gitbook.io/spirit-swap/tokenomics/fees

In [82]:
query = gql('''
query {
  spiritswapFactories{
    pairCount
  }
}
''')

response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(query)
number_of_pairs = int(response['spiritswapFactories'][0]['pairCount'])
print(number_of_pairs)

764


In [83]:
try:
    df = pd.read_excel('pairdata_Spiritswap_FTM.xlsx')
except:
    df = pd.DataFrame([])
    response_dict = {}
    response_list = []


    for i in range(number_of_pairs):
        if i % 100 == 0:
            querystring = '''
                  query {
                  pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                          token0{symbol}
                          token1{symbol}
                          id
                      }
                    }
                    '''
            try:
                response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
                df = df.append(pd.DataFrame.from_dict(response['pairs']))
                print(len(df))
            except:
                pass

In [84]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    df.head(2)

In [85]:
criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.to_excel("pairdata_Spiritswap_FTM.xlsx", index = False)
df_filtered.head(2)

,id,Pair,Token 0,Token 1
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,WFTM-CREAM,WFTM,CREAM
1,0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,WFTM-FRAX,WFTM,FRAX


In [86]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,id,timestamp,volumeUSD
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-22 06:45:25.814126,4.596235e+06


In [87]:
df_old = pd.read_csv("Spiritswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-20 07:38:05.289675,4.582181e+06


In [88]:
df_timedelta.to_csv("Spiritswap export FTM.csv", index = False)

In [89]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(2)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,2021-07-22 06:45:25.814126,4.596235e+06,2021-07-20 07:38:05.289675,4.582181e+06,0.005376,14053.966562,7843.061373
0x0ec0e1629e776272fa3e55548d4a656be0eedcf4,2021-07-22 06:45:26.153358,1.109483e+07,2021-07-20 07:38:05.488801,1.101446e+07,0.005376,80363.606696,44848.276544


In [90]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
              query {
  pairDayDatas(where: {pairAddress:"''' + id + '''"} orderBy: date, orderDirection: desc first: 2) {
    date
    dailyVolumeUSD
    reserveUSD
  }
}'''
    response = client('https://api.thegraph.com/subgraphs/name/layer3org/spiritswap-analytics').execute(gql(querystring))
    response
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a
 2 / 21 pair: 0x0ec0e1629e776272fa3e55548d4a656be0eedcf4
 3 / 21 pair: 0x13bac72a9b0d081aa3926ee7f98d0e1b21dd7d73
 4 / 21 pair: 0x279b2c897737a50405ed2091694f225d83f2d3ba
 5 / 21 pair: 0x33068ad825fbeedb0e57e0466e379290b699f57c
 6 / 21 pair: 0x4556f6afa9b016a83052d459290573a5d83c8764
 7 / 21 pair: 0x4fc38a2735c7da1d71ccabf6dec235a7da4ec52c
 8 / 21 pair: 0x5427e7ab61145c41b1e58453121245daef3929c3
 9 / 21 pair: 0x613bf4e46b4817015c01c6bb31c7ae9edaadc26e
10 / 21 pair: 0x6a65a67da11b68cd141581483a9f7a0cb3c47bae
11 / 21 pair: 0x6d134d63417dc67f57dbc9fcaad4b4cbfab3af2f
12 / 21 pair: 0x6d87359b1e3fd852bd36250018e3328d1afd78e7
13 / 21 pair: 0x9606d683d03f012dda296ef0ae9261207c4a5847
14 / 21 pair: 0x9fe4c0ce5f533e96c2b72d852f190961ad5a7bb3
15 / 21 pair: 0xb67647130d0d51b5a0d53e3ae99ea24ca6100ebb
16 / 21 pair: 0xd061c6586670792331e14a80f3b3bb267189c681
17 / 21 pair: 0xd14dd3c56d9bc306322d4cea0e1c49e9ddf045d4
18 / 21 pair: 0xd3a0c65e5cefcee

,volumeUSD,date,reserveUSD,id
0,11.299064,2021-07-22,188738.518896,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a


In [91]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpiritSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,11.299064,2021-07-22,188738.518896,0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,[SpiritSwap FTM] WFTM-CREAM,WFTM,CREAM,0.033897,0.007,0.007,0.007,0.007,0.006,0.007,0.007,0.007,0.007


In [92]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xd52c57594f1b62cba87dbf079608332abe96e02e,234.171903,2021-07-21,4426.954479,[SpiritSwap FTM] USDC-FRAX,0.702516,5.792,5.542,5.204,4.725,3.99,6.066,6.53,7.482,10.565


In [93]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)
df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x040dd0d0f5e2a01feb0c5457abb588b23cf4c43a,4.155517,4.151118,4.144537,4.133616,4.111944,4.159925,4.166555,4.177652,4.200023,188738.518896


In [94]:
# df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
#                                  axis = 1)[['Pair',
#                                             'date',
#                                             'APR timedelta',
#                                             'APR timedelta Liq + 200',
#                                             'APR timedelta Liq + 500',
#                                             'APR timedelta Liq + 1000',                                            
#                                             'APR timedelta Liq + 2000',
#                                             'APR timedelta Liq - 200',
#                                             'APR timedelta Liq - 500',
#                                             'APR timedelta Liq - 1000',                                            
#                                             'APR timedelta Liq - 2000',                                             
#                                             'APR yday',
#                                             'APR yday Liq + 200',
#                                             'APR yday Liq + 500',
#                                             'APR yday Liq + 1000',                                            
#                                             'APR yday Liq + 2000',
#                                             'APR yday Liq - 200',
#                                             'APR yday Liq - 500',
#                                             'APR yday Liq - 1000',                                            
#                                             'APR yday Liq - 2000',                                            
#                                             'reserveUSD',
#                                             'volumeUSD']].set_index('Pair')\
#                        .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

# df_final_spiritswap.columns = ['date',
#                               'APR timedelta',
#                                 'APR timedelta',
#                                 'APR timedelta Liq + 200',
#                                 'APR timedelta Liq + 500',
#                                 'APR timedelta Liq + 1000',                                            
#                                 'APR timedelta Liq + 2000',
#                                 'APR timedelta Liq - 200',
#                                 'APR timedelta Liq - 500',
#                                 'APR timedelta Liq - 1000',                                            
#                                 'APR timedelta Liq - 2000',     
#                               'APR yday',
#                                 'APR yday Liq + 200',
#                                 'APR yday Liq + 500',
#                                 'APR yday Liq + 1000',                                            
#                                 'APR yday Liq + 2000',
#                                 'APR yday Liq - 200',
#                                 'APR yday Liq - 500',
#                                 'APR yday Liq - 1000',                                            
#                                 'APR yday Liq - 2000', 
#                               'Liquidity timedelta',
#                               'Liquidity yday',
#                               'Volume yday']

# df_final_spiritswap.head(1)

df_final_spiritswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                                 
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spiritswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                                    
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                                    
                              'APR timedelta Liq - 2000',                                    
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',
                              'APR yday Liq - 2000',                                     
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

## SpookySwap

In [95]:
client_exe = client('https://api.thegraph.com/subgraphs/name/eerieeight/spooky-swap-exchange')
fee = 0.002 # treasury gets 0, lps 0.2%, see https://docs.spookyswap.finance/products/exchange#swap-faq 

In [96]:
query = gql('''
            query {uniswapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['uniswapFactories'][0]['pairCount'])
print(number_of_pairs)

626


In [97]:
# try:
#     df = pd.read_excel('pairdata_Spookyswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []

for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
          query {
          pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                  token0 {symbol}
                  token1 {symbol}
                  id
              }
            }
            '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass

100
200
300
354
354
354
354


In [98]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
8,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,WFTM-SNX,WFTM,SNX


In [99]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,id,timestamp,volumeUSD
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-22 06:45:39.845745,6.580478e+06


In [100]:
df_old = pd.read_csv("Spookyswap export.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-20 07:38:24.397611,6.421861e+06


In [101]:
df_timedelta.to_csv("Spookyswap export.csv", index = False)

In [102]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,2021-07-22 06:45:39.845745,6.580478e+06,2021-07-20 07:38:24.397611,6.421861e+06,0.005376,158617.292506,59014.531939


In [103]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 21 pair: 0x06d173628be105fe81f1c82c9979ba79ebcafcb7
 2 / 21 pair: 0x2b4c76d0dc16be1c31d4c1dc53bf9b45987fc75c
 3 / 21 pair: 0x484237bc35ca671302d19694c66d617142fbc235
 4 / 21 pair: 0x4a217fbf837c6115b183eaac0245d6a72e8a98e4
 5 / 21 pair: 0x5965e53aa80a0bcf1cd6dbdd72e6a9b2aa047410
 6 / 21 pair: 0x5dc7848bf215f1d99f2af3d2bf78fcdf238ee34b
 7 / 21 pair: 0x7fe139289e3837f8e6d352ecd013c6dbfd903a64
 8 / 21 pair: 0x89d9bc2f2d091cfbfc31e333d6dc555ddbc2fd29
 9 / 21 pair: 0x91b39d5584e2a7dc829f696235742cc293f2e8cf
10 / 21 pair: 0xa572bdf049382f1f98f9a430788dadd51a303969
11 / 21 pair: 0xb77b223490e1f5951ec79a8d09db9eab2adcb934
12 / 21 pair: 0xbf4d61d4cec3a9dff7452d8987e1cc2943e2eb4c
13 / 21 pair: 0xd92206379bd8203ac38225af006bb96bf1f12412
14 / 21 pair: 0xde1a99d2471470c33d9130c5dd485b0c99f3d0df
15 / 21 pair: 0xe120ffbda0d14f3bb6d6053e90e63c572a66a428
16 / 21 pair: 0xebf374bb21d83cf010cc7363918776adf6ff2bf6
17 / 21 pair: 0xec454eda10accdd66209c57af8c12924556f3abd
18 / 21 pair: 0xf0702249f4d3a25

,volumeUSD,date,reserveUSD,id
0,8622.537948,2021-07-22,792380.958317,0x06d173628be105fe81f1c82c9979ba79ebcafcb7


In [104]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SpookySwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,volumeUSD,date,reserveUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,8622.537948,2021-07-22,792380.958317,0x06d173628be105fe81f1c82c9979ba79ebcafcb7,[SpookySwap FTM] WFTM-SNX,WFTM,SNX,17.245076,0.794,0.794,0.794,0.793,0.792,0.795,0.795,0.795,0.796


In [105]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,volumeUSD,date,reserveUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0xe120ffbda0d14f3bb6d6053e90e63c572a66a428,653447.3127,2021-07-21,2.335870e+06,[SpookySwap FTM] WFTM-DAI,1306.894625,20.421,20.42,20.417,20.413,20.404,20.423,20.426,20.43,20.439


In [106]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x06d173628be105fe81f1c82c9979ba79ebcafcb7,7.447747,7.445868,7.443051,7.43836,7.428996,7.449628,7.45245,7.457158,7.466593,792380.958317


In [107]:
df_final_spookyswap = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_spookyswap.columns = ['date',
                              'APR timedelta',
                              'APR timedelta Liq + 200',
                              'APR timedelta Liq + 500',
                              'APR timedelta Liq + 1000',                               
                              'APR timedelta Liq + 2000',
                              'APR timedelta Liq - 200',
                              'APR timedelta Liq - 500',
                              'APR timedelta Liq - 1000',                               
                              'APR timedelta Liq - 2000',                               
                              'APR yday',
                              'APR yday Liq + 200',
                              'APR yday Liq + 500',
                              'APR yday Liq + 1000',                               
                              'APR yday Liq + 2000',
                              'APR yday Liq - 200',
                              'APR yday Liq - 500',
                              'APR yday Liq - 1000',                               
                              'APR yday Liq - 2000',                               
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_spookyswap.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[SpookySwap FTM] USDC-WFTM,2021-07-21,43.675028,43.672779,43.669406,43.663785,43.652548,43.677277,43.680651,43.686276,43.697531,44.638,44.635,44.632,44.626,44.615,44.64,44.643,44.649,44.661,3.883709e+06,3.897244e+06,2.383070e+06


## SushiSwap FTM

In [108]:
client_exe = client('https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange')
fee = 0.0025

In [109]:
query = gql('''
            query {factories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

735


In [110]:
# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

100
183
183
183
183
183
183
183


,id,token0,token1
0,0x02f07a24ec8e6b34514f2cc8282af7d67f55d010,{'symbol': 'WFTM'},{'symbol': 'eBURN'}


In [111]:
if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

,id,Pair,Token 0,Token 1
16,0x147e27be8e66f74c7659d88e2e6a13188873d58b,WFTM-CREAM,WFTM,CREAM


In [112]:
df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

 1 / 18 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 18 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 18 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 18 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 18 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 18 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 7 / 18 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
 8 / 18 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
 9 / 18 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
10 / 18 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
11 / 18 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
12 / 18 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
13 / 18 pair: 0xc92d97416328a461bce2f539498b505c6c97748f
14 / 18 pair: 0xd019dd7c760c6431797d6ed170bffb8faee11f99
15 / 18 pair: 0xd32f2eb49e91aa160946f3538564118388d6246a
16 / 18 pair: 0xe74a6b1f2034c8c137610cafad49a6763cd25865
17 / 18 pair: 0xed88d84c8728baaa53fd7226c9e55bd36e777ade
18 / 18 pair: 0xfe69403cf2e2239

,id,timestamp,volumeUSD
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-22 06:46:16.337135,2.420705e+06


In [113]:
df_old = pd.read_csv("Sushiswap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

,id_old,timestamp_old,volumeUSD_old
0,0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-20 07:38:58.163280,2.415023e+06


In [114]:
df_timedelta.to_csv("Sushiswap export FTM.csv", index = False)

In [115]:
df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

,timestamp,volumeUSD,timestamp_old,volumeUSD_old,timedelta,volumeUSD delta,fees per year
0x147e27be8e66f74c7659d88e2e6a13188873d58b,2021-07-22 06:46:16.337135,2.420705e+06,2021-07-20 07:38:58.163280,2.415023e+06,0.005376,5682.848325,3171.456655


In [116]:
df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pair:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            volumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 18 pair: 0x147e27be8e66f74c7659d88e2e6a13188873d58b
 2 / 18 pair: 0x1ca86e57103564f47ffcea7259a6ce8cc1301549
 3 / 18 pair: 0x27884c7647deb61c9f2b9202d1141bb84661756b
 4 / 18 pair: 0x280abc0d9727f4de3a46ca9cf93c16b4a9c127d1
 5 / 18 pair: 0x2d89bb3b1448152002453aa37fbe4051afdbf56d
 6 / 18 pair: 0x3d0bd54c48c2c433ea6fed609cc3d5fb7a77622b
 7 / 18 pair: 0x49d2e0dc99c7358d7a9a8633bf6df111d0ee7f65
 8 / 18 pair: 0x6aafe06996ac5b5d504ce7ba90dfdd5393679f0d
 9 / 18 pair: 0x8be92f3d64e91d08ab1cc8a5c487da3f1695b11e
10 / 18 pair: 0xa48869049e36f8bfe0cc5cf655632626988c0140
11 / 18 pair: 0xafa1bf9b2a89dfd60499838c331538283b3346ed
12 / 18 pair: 0xbd0eec56da621b6b31e3d06614f7853624e1c0af
13 / 18 pair: 0xc92d97416328a461bce2f539498b505c6c97748f
14 / 18 pair: 0xd019dd7c760c6431797d6ed170bffb8faee11f99
15 / 18 pair: 0xd32f2eb49e91aa160946f3538564118388d6246a
16 / 18 pair: 0xe74a6b1f2034c8c137610cafad49a6763cd25865
17 / 18 pair: 0xed88d84c8728baaa53fd7226c9e55bd36e777ade
18 / 18 pair: 0xfe69403cf2e2239

,date,reserveUSD,volumeUSD,id
0,2021-07-22,109062.334531,6.856652,0x147e27be8e66f74c7659d88e2e6a13188873d58b


In [117]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[SushiSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

,date,reserveUSD,volumeUSD,id,Pair,Token 0,Token 1,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
0,2021-07-22,109062.334531,6.856652,0x147e27be8e66f74c7659d88e2e6a13188873d58b,[SushiSwap FTM] WFTM-CREAM,WFTM,CREAM,0.02057,0.007,0.007,0.007,0.007,0.007,0.007,0.007,0.007,0.007


In [118]:
df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

,date,reserveUSD,volumeUSD,Pair,fee,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000
id,,,,,,,,,,,,,,
0x2d89bb3b1448152002453aa37fbe4051afdbf56d,2021-07-21,75965.75754,9806.209308,[SushiSwap FTM] USDC-AAVE,29.418628,14.135,14.098,14.043,13.951,13.772,14.172,14.229,14.324,14.517


In [119]:
# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,reserveUSD
0x147e27be8e66f74c7659d88e2e6a13188873d58b,2.90793,2.902607,2.89466,2.88151,2.855565,2.913273,2.921323,2.93484,2.962252,109062.334531


In [120]:
df_final_sushiswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_sushiswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_sushiswap_ftm.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[SushiSwap FTM] USDC-WFTM,2021-07-21,33.771482,33.650736,33.471228,33.176265,32.601665,33.893098,34.077173,34.388448,35.028375,36.557,36.429,36.238,35.924,35.313,36.686,36.882,37.213,37.892,55738.051756,56754.78197,18947.845995


## WakaSwap FTM

In [121]:
client_exe = client('https://api.thegraph.com/subgraphs/name/waka-finance/waka-graph') # https://thegraph.com/explorer/subgraph/waka-finance/waka-graph
fee = 0.002 # 0.3 fee, 0.2 to LPs, 0.1 to stakers https://waka-finance.medium.com/welcome-to-waka-finance-a-native-defi-hub-powered-by-fantom-opera-home-of-the-super-fast-e3d75d3a64f6

query = gql('''
            query {wakaSwapFactories {pairCount}}
            ''')

response = client_exe.execute(query)
number_of_pairs = int(response['wakaSwapFactories'][0]['pairCount'])
print(number_of_pairs)

# try:
#     df = pd.read_excel('pairdata_Sushiswap_FTM.xlsx')
# except:
df = pd.DataFrame([])
response_dict = {}
response_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      token0 {symbol}
                      token1 {symbol}
                      id
                  }
                }
                '''
        try:
            response = client_exe.execute(gql(querystring))
            df = df.append(pd.DataFrame.from_dict(response['pairs']))
            print(len(df))
        except:
            pass
            
df.head(1)

if df.columns.tolist() == ['id','Pair', 'Token 0', 'Token 1']:
    pass
else:
    df['token0'] = df['token0'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['token1'] = df['token1'].astype(str).str.replace("{'symbol': ",'', regex= True)\
                                               .replace("}",'', regex= True)\
                                               .replace("'",'', regex= True)
    df['Pair'] = df['token0'] + '-' + df['token1']
    df.columns = ['id','Token 0', 'Token 1', 'Pair']
    df = df[['id','Pair', 'Token 0', 'Token 1']]
    criteria = df['Token 0'].isin(interesting_tokens) & \
               df['Token 1'].isin(interesting_tokens)

    df_filtered = df.loc[criteria,:]
df_filtered.head(1)

df_timedelta = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')

    querystring = '''
                  query {
      pairs(where: {id:"''' + id + '''"}){
        volumeUSD
      }
    }'''
    response = client_exe.execute(gql(querystring))

    df1 = pd.DataFrame.from_dict(response['pairs'])
    df1['id'] = id
    df1['timestamp'] = datetime.now()
    df1['volumeUSD'] = df1['volumeUSD'].astype(float)
    df_timedelta = df_timedelta.append(df1)
    
df_timedelta = df_timedelta[['id', 'timestamp', 'volumeUSD']]
df_timedelta.head(1)

38
24
 1 / 13 pair: 0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb
 2 / 13 pair: 0x195c45044b121d8caf4c9e2d8f47fb81f4e12f21
 3 / 13 pair: 0x1b084cda6e11347b3c15eae537cf8a7ce39f96da
 4 / 13 pair: 0x2e783ce36830a6274761cc4a9d04d275553f3bc1
 5 / 13 pair: 0x32736ae178d0debf137326a0c0c6d50e374d9f19
 6 / 13 pair: 0x3c4ad49f26b47f00f7e5165b942d41367dd60a04
 7 / 13 pair: 0x60ff8c1dffc8252cfd94c07f65feb4ce90ee9b99
 8 / 13 pair: 0x68a242300e9547263aa8d57c1e4a7d75cf3c8e5f
 9 / 13 pair: 0x78da2bd2c18837c9d1b9008c81078e378f1952c6
10 / 13 pair: 0x97a490e7c90b4d8f6d18752672f4c8e3c35891a2
11 / 13 pair: 0x9c58df352fcd984700c57324a116845378adc5ba
12 / 13 pair: 0xb0e5d65b4c22ff9f099e449df03acb3a57bb5f27
13 / 13 pair: 0xff09b99aed8fb58c209904f6adc4a1fcceea79ac


,id,timestamp,volumeUSD
0,0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb,2021-07-22 06:46:29.327742,7.260421e+06


In [122]:
df_old = pd.read_csv("WakaSwap export FTM.csv",
                     usecols = ['id', 'timestamp', 'volumeUSD'])
df_old = df_old[['id', 'timestamp', 'volumeUSD']]
df_old['timestamp'] = pd.to_datetime(df_old['timestamp'])
df_old.columns = ['id_old', 'timestamp_old', 'volumeUSD_old']
df_old.head(1)

df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

df_combined = pd.concat([df_timedelta.set_index('id'), df_old.set_index('id_old')], axis = 1)
df_combined['timedelta'] = (df_combined['timestamp'] - df_combined['timestamp_old']) / np.timedelta64(1, 'Y')
df_combined['volumeUSD delta'] = df_combined['volumeUSD'] - df_combined['volumeUSD_old']
df_combined['fees per year'] = df_combined['volumeUSD delta'] * fee / df_combined['timedelta']

df_combined.head(1)

df_last_two_days = pd.DataFrame([])
lenid = len(df_filtered['id'])

for count, id in enumerate(df_filtered['id']):
    print(f'{count+1:2} / {lenid} pair: {id}')
        
    querystring = '''
                  query {
                         pairDayDatas(where: {pairAddress:"''' + id + '''"}
                                      orderBy: date
                                      orderDirection: desc 
                                      first: 2) 
                                      {
                            date
                            dailyVolumeUSD
                            reserveUSD}
                        }
                  '''
    response = client_exe.execute(gql(querystring))
    
    df1 = pd.DataFrame.from_dict(response['pairDayDatas'])
    df1['id'] = id
    df_last_two_days = df_last_two_days.append(df1)
    
df_last_two_days = df_last_two_days.rename(columns = {'dailyVolumeUSD':'volumeUSD'})
df_last_two_days[['volumeUSD','reserveUSD']] = df_last_two_days[['volumeUSD','reserveUSD']].astype(float)
df_last_two_days['date'] = pd.to_datetime(df_last_two_days['date'], unit='s').dt.date
df_last_two_days.head(1)

 1 / 13 pair: 0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb
 2 / 13 pair: 0x195c45044b121d8caf4c9e2d8f47fb81f4e12f21
 3 / 13 pair: 0x1b084cda6e11347b3c15eae537cf8a7ce39f96da
 4 / 13 pair: 0x2e783ce36830a6274761cc4a9d04d275553f3bc1
 5 / 13 pair: 0x32736ae178d0debf137326a0c0c6d50e374d9f19
 6 / 13 pair: 0x3c4ad49f26b47f00f7e5165b942d41367dd60a04
 7 / 13 pair: 0x60ff8c1dffc8252cfd94c07f65feb4ce90ee9b99
 8 / 13 pair: 0x68a242300e9547263aa8d57c1e4a7d75cf3c8e5f
 9 / 13 pair: 0x78da2bd2c18837c9d1b9008c81078e378f1952c6
10 / 13 pair: 0x97a490e7c90b4d8f6d18752672f4c8e3c35891a2
11 / 13 pair: 0x9c58df352fcd984700c57324a116845378adc5ba
12 / 13 pair: 0xb0e5d65b4c22ff9f099e449df03acb3a57bb5f27
13 / 13 pair: 0xff09b99aed8fb58c209904f6adc4a1fcceea79ac


,volumeUSD,date,reserveUSD,id
0,335.873804,2021-07-22,74146.107171,0x0eea1bf6342c71171882c2ac3838a9c78e00b5cb


In [123]:
df_timedelta.to_csv("WakaSwap export FTM.csv", index = False)

In [124]:
df_last_two_days_incl_pairs = df_last_two_days.merge(df_filtered,
                                                     left_on = 'id',
                                                     right_on = 'id',
                                                     how = 'left',
                                                     validate = 'many_to_one')

df_last_two_days_incl_pairs['fee'] = round(df_last_two_days_incl_pairs['volumeUSD'] * fee, 6)
df_last_two_days_incl_pairs['APR yday'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / df_last_two_days_incl_pairs['reserveUSD'], 3)
df_last_two_days_incl_pairs['Pair'] = '[WakaSwap FTM] ' + df_last_two_days_incl_pairs['Pair']
df_last_two_days_incl_pairs['APR yday Liq + 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 200), 3)
df_last_two_days_incl_pairs['APR yday Liq + 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 500), 3)
df_last_two_days_incl_pairs['APR yday Liq + 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq + 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] + 2000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 200'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 200), 3)
df_last_two_days_incl_pairs['APR yday Liq - 500'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 500), 3)
df_last_two_days_incl_pairs['APR yday Liq - 1000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 1000), 3)
df_last_two_days_incl_pairs['APR yday Liq - 2000'] = round(100 * df_last_two_days_incl_pairs['fee'] * 365 / (df_last_two_days_incl_pairs['reserveUSD'] - 2000), 3)

df_last_two_days_incl_pairs.head(1)

df_yesterday = df_last_two_days_incl_pairs.sort_values(by = 'date',
                                                       ascending = False)\
                                          .drop_duplicates(subset = ['Pair'], keep = 'last')\
                                          .reset_index(drop = True)\
                                          .drop(columns = ['Token 0', 'Token 1'])\
                                          .set_index('id')

df_yesterday.head(1)

# add in reserveUSD to calculate timedelta APYs

df_combined2 = pd.concat([df_combined,
                          df_last_two_days_incl_pairs.sort_values(by = 'date', ascending = False)\
                                                     .drop_duplicates(subset = ['Pair'],
                                                                      keep = 'first')\
                                                     .set_index('id')['reserveUSD']],
                         axis = 1)

df_combined2['APR timedelta'] = 100 * df_combined2['fees per year'] / df_combined2['reserveUSD']
df_combined2['APR timedelta Liq + 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 200)
df_combined2['APR timedelta Liq + 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 500)
df_combined2['APR timedelta Liq + 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 1000)
df_combined2['APR timedelta Liq + 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] + 2000)
df_combined2['APR timedelta Liq - 200'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 200)
df_combined2['APR timedelta Liq - 500'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 500)
df_combined2['APR timedelta Liq - 1000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 1000)
df_combined2['APR timedelta Liq - 2000'] = 100 * df_combined2['fees per year'] / (df_combined2['reserveUSD'] - 2000)

df_combined2 = df_combined2[['APR timedelta',
                             'APR timedelta Liq + 200',
                             'APR timedelta Liq + 500',
                             'APR timedelta Liq + 1000',                             
                             'APR timedelta Liq + 2000',
                             'APR timedelta Liq - 200',
                             'APR timedelta Liq - 500',
                             'APR timedelta Liq - 1000',                             
                             'APR timedelta Liq - 2000',                             
                             'reserveUSD']]

df_combined2.head(1)

df_final_wakaswap_ftm = pd.concat([df_combined2, df_yesterday],
                                 axis = 1)[['Pair',
                                            'date',
                                            'APR timedelta',
                                            'APR timedelta Liq + 200',
                                            'APR timedelta Liq + 500',
                                            'APR timedelta Liq + 1000',                                            
                                            'APR timedelta Liq + 2000',
                                            'APR timedelta Liq - 200',
                                            'APR timedelta Liq - 500',
                                            'APR timedelta Liq - 1000',                                            
                                            'APR timedelta Liq - 2000',                                            
                                            'APR yday',
                                            'APR yday Liq + 200',
                                            'APR yday Liq + 500',
                                            'APR yday Liq + 1000',                                            
                                            'APR yday Liq + 2000',
                                            'APR yday Liq - 200',
                                            'APR yday Liq - 500',
                                            'APR yday Liq - 1000',                                            
                                            'APR yday Liq - 2000',                                            
                                            'reserveUSD',
                                            'volumeUSD']].set_index('Pair')\
                       .sort_values(by = 'APR timedelta Liq + 2000', ascending = False)

df_final_wakaswap_ftm.columns = ['date',
                              'APR timedelta',
                                'APR timedelta Liq + 200',
                                    'APR timedelta Liq + 500',
                                    'APR timedelta Liq + 1000',                                            
                                    'APR timedelta Liq + 2000',
                                    'APR timedelta Liq - 200',
                                    'APR timedelta Liq - 500',
                                    'APR timedelta Liq - 1000',                                            
                                    'APR timedelta Liq - 2000',   
                              'APR yday',
                                'APR yday Liq + 200',
                                    'APR yday Liq + 500',
                                    'APR yday Liq + 1000',                                            
                                    'APR yday Liq + 2000',
                                    'APR yday Liq - 200',
                                    'APR yday Liq - 500',
                                    'APR yday Liq - 1000',                                            
                                    'APR yday Liq - 2000',  
                              'Liquidity timedelta',
                              'Liquidity yday',
                              'Volume yday']

df_final_wakaswap_ftm.head(1)

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[WakaSwap FTM] USDC-WFTM,2021-07-21,23.252762,23.217706,23.165319,23.07853,22.906889,23.287925,23.340869,23.429646,23.609241,22.963,22.929,22.878,22.792,22.624,22.998,23.05,23.137,23.313,132458.260354,133286.203262,27951.745264


In [125]:
pd.set_option('display.max_columns', None)

df_final_combined = df_final_sushiswap_matic.append(df_final_quickswap)\
                                            .append(df_final_honeyswap_matic)\
                                            .append(df_final_wakaswap_ftm)\
                                            .append(df_final_spiritswap)\
                                            .append(df_final_spookyswap)\
                                            .append(df_final_sushiswap_ftm)\
                                            .append(df_final_DoDoSwap_matic)\
                                            .append(df_final_apeswap_matic)\
                                            .sort_values(by = 'APR timedelta Liq + 200',
                                                         ascending = False)

df_final_combined[['APR timedelta',
                   'APR timedelta Liq + 200',
                   'APR timedelta Liq + 500',
                   'APR timedelta Liq + 1000',                   
                   'APR timedelta Liq + 2000',
                   'APR timedelta Liq - 200',
                   'APR timedelta Liq - 500',
                   'APR timedelta Liq - 1000',                   
                   'APR timedelta Liq - 2000',                   
                   'APR yday',
                   'APR yday Liq + 200',
                   'APR yday Liq + 500',
                   'APR yday Liq + 1000',                   
                   'APR yday Liq + 2000',
                   'APR yday Liq - 200',
                   'APR yday Liq - 500',
                   'APR yday Liq - 1000',                   
                   'APR yday Liq - 2000']] = df_final_combined[['APR timedelta',
                                                                'APR timedelta Liq + 200',
                                                               'APR timedelta Liq + 500',
                                                               'APR timedelta Liq + 1000',                   
                                                               'APR timedelta Liq + 2000',
                                                               'APR timedelta Liq - 200',
                                                               'APR timedelta Liq - 500',
                                                               'APR timedelta Liq - 1000',                   
                                                               'APR timedelta Liq - 2000',                       
                                                                'APR yday',
                                                                'APR yday Liq + 200',
                                                               'APR yday Liq + 500',
                                                               'APR yday Liq + 1000',                   
                                                               'APR yday Liq + 2000',
                                                               'APR yday Liq - 200',
                                                               'APR yday Liq - 500',
                                                               'APR yday Liq - 1000',                   
                                                               'APR yday Liq - 2000']].round(2)

df_final_combined[['Liquidity timedelta',
                   'Liquidity yday',
                   'Volume yday']] = df_final_combined[['Liquidity timedelta',
                                                        'Liquidity yday',
                                                        'Volume yday']].fillna(0).astype(int)

df_final_combined['APR timedelta'] = df_final_combined['APR timedelta'].round(1)

criteria = (df_final_combined['Volume yday'] > 0) & (df_final_combined['APR timedelta'] > 0)

df_final_combined.loc[criteria, :].dropna(subset = ['date'])

,date,APR timedelta,APR timedelta Liq + 200,APR timedelta Liq + 500,APR timedelta Liq + 1000,APR timedelta Liq + 2000,APR timedelta Liq - 200,APR timedelta Liq - 500,APR timedelta Liq - 1000,APR timedelta Liq - 2000,APR yday,APR yday Liq + 200,APR yday Liq + 500,APR yday Liq + 1000,APR yday Liq + 2000,APR yday Liq - 200,APR yday Liq - 500,APR yday Liq - 1000,APR yday Liq - 2000,Liquidity timedelta,Liquidity yday,Volume yday
Pair,,,,,,,,,,,,,,,,,,,,,,
[Sushiswap MATIC] USDC-LINK,2021-07-21,578.3,266.55,147.38,84.45,45.55,-3412.71,-300.62,-119.30,-54.07,604.42,277.95,153.54,87.94,47.42,-3461.80,-312.12,-124.04,-56.25,171,170,939
[ApeSwap MATIC] WMATIC-USDC,2021-07-21,146.0,130.41,112.37,91.31,66.42,165.95,208.56,364.65,-734.06,92.48,82.56,71.12,57.78,42.01,105.11,132.18,231.62,-459.12,1668,1664,2108
[Sushiswap MATIC] WMATIC-USDT,2021-07-21,98.3,96.56,94.12,90.31,83.55,100.03,102.79,107.76,119.28,114.48,112.47,109.59,105.10,97.14,116.56,119.82,125.70,139.35,11353,11205,11715
[Sushiswap MATIC] WMATIC-QUICK,2021-07-21,113.9,89.80,68.14,48.61,30.89,155.81,347.32,-331.27,-67.50,120.94,95.20,72.16,51.42,32.65,165.76,373.30,-343.52,-70.97,744,739,816
[Sushiswap MATIC] WMATIC-USDC,2021-07-21,88.2,88.14,88.14,88.13,88.10,88.15,88.16,88.17,88.19,103.66,103.66,103.65,103.64,103.61,103.67,103.67,103.69,103.71,3999355,3984404,3771951
[Quickswap MATIC] WMATIC-USDT,2021-07-21,80.4,80.34,80.30,80.23,80.09,80.40,80.44,80.51,80.65,100.97,100.94,100.88,100.80,100.62,101.01,101.06,101.15,101.33,574136,571499,526989
[Quickswap MATIC] WMATIC-USDC,2021-07-21,76.2,76.20,76.20,76.20,76.19,76.20,76.20,76.20,76.20,92.65,92.65,92.65,92.65,92.65,92.66,92.66,92.66,92.66,28324077,27683769,23424775
[Sushiswap MATIC] WMATIC-WBTC,2021-07-21,69.8,68.61,66.95,64.36,59.74,70.95,72.81,76.14,83.81,64.75,63.68,62.14,59.73,55.43,65.86,67.60,70.70,77.85,11926,11885,7028
[Sushiswap MATIC] WMATIC-DAI,2021-07-21,65.6,64.55,62.95,60.46,56.03,66.80,68.60,71.82,79.27,64.02,62.93,61.37,58.94,54.60,65.14,66.90,70.06,77.36,11642,11595,6778
